In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sentence_transformers import SentenceTransformer



In [46]:
df = pd.read_json("../data/raw/nytimes_headlines.json")
df = df.rename(columns={0: "headline"})
df = df.drop(df[df.headline.str.contains("No Title")].index).reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12564 entries, 0 to 12563
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     12564 non-null  int64 
 1   headline  12564 non-null  object
dtypes: int64(1), object(1)
memory usage: 196.4+ KB


In [47]:
df.head()

,index,headline
0,0,"STOCK FLOTATIONS SURGED IN MONTH; $80,649,000 ..."
1,1,Italy Reprieves Slot Machines
2,2,Mayor Going to Albany For Inauguration Today
3,3,Three Tie for First Place
4,4,Imperial Oil Aide on Board


In [48]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df.headline)

In [20]:
df_e = pd.DataFrame(embeddings)
df_e.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.009170,-0.032507,-0.023817,0.105160,0.010962,-0.023618,-0.013871,0.107328,-0.041880,-0.044648,...,0.034181,0.019903,-0.022057,0.041632,-0.033228,-0.051318,-0.007030,-0.198016,-0.038070,0.063735
1,-0.088845,0.033788,-0.055447,0.036071,-0.112650,-0.001982,0.023443,-0.005132,0.001620,-0.043282,...,0.063477,0.042395,0.040532,0.077330,0.094920,0.061836,-0.001583,-0.006795,-0.020796,-0.000644
2,-0.034970,0.090391,0.109762,-0.024841,-0.078084,0.010870,0.073226,0.005116,-0.106012,0.000121,...,0.015394,-0.017323,-0.101041,0.067641,-0.017842,0.020313,0.007027,-0.025982,0.000882,-0.027439
3,-0.002881,-0.002144,-0.086286,-0.038686,-0.005081,0.052005,-0.005780,-0.014854,0.078658,-0.007532,...,0.057623,0.044072,-0.021265,0.025061,-0.094952,0.056536,-0.036046,-0.047084,0.037562,0.012289
4,-0.049082,-0.047072,0.018786,0.033728,0.009560,0.027688,-0.026062,0.061782,-0.032944,-0.098341,...,0.017256,-0.051608,-0.064796,-0.002627,-0.014820,-0.006872,-0.072870,-0.036834,-0.036593,-0.010620


In [22]:
from sklearn.cluster import AgglomerativeClustering

In [49]:
cmodel = AgglomerativeClustering(n_clusters=None, metric='cosine', linkage='average', distance_threshold=0.2)
a_clusters = cmodel.fit(embeddings)

In [50]:

pd.Series(a_clusters.labels_).value_counts().head(5)

324    31
259    27
171    26
152    25
264    23
Name: count, dtype: int64

In [53]:
df[a_clusters.labels_ == 171]

,index,headline
331,345,MUSIC NOTES
722,757,MUSIC NOTES
1690,1764,MUSIC NOTES
1943,2026,MUSIC NOTES
2473,2570,MUSIC NOTES
2676,2781,MUSIC NOTES
3254,3379,MUSIC NOTES
3485,3622,MUSIC NOTES
4310,4472,MUSIC NOTES
4948,5130,MUSIC NOTES


In [55]:
nyt = pd.read_json("../data/raw/nytimes_articles.json")
nyt.columns

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_page',
       'source', 'multimedia', 'headline', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'byline',
       'type_of_material', '_id', 'word_count', 'uri', 'print_section'],
      dtype='object')

In [88]:
nyt['main_headline'] = nyt.headline.map(lambda x: x["main"])

In [89]:
emodel = SentenceTransformer("all-MiniLM-L6-v2")
abstract_embeddings = emodel.encode(nyt.main_headline + ' - ' + nyt.abstract)

In [110]:
ac_model= AgglomerativeClustering(n_clusters=None, metric='cosine', linkage='average', distance_threshold=0.4)
abstract_clusters = ac_model.fit(abstract_embeddings)

In [111]:
ac_model.n_clusters_

10162

In [112]:
df_labels = pd.Series(ac_model.labels_)
df_labels.value_counts()

356      337
1190      57
66        36
556       28
1169      25
        ... 
8521       1
10011      1
7546       1
6664       1
1755       1
Name: count, Length: 10162, dtype: int64

In [118]:
nyt[df_labels == 8521].headline 

4565    {'main': 'ROSY GLOW TINTS EARNINGS PICTURE; Co...
Name: headline, dtype: object